In [1]:
import json
import requests
import statistics as st
import pandas as pd
import numpy as np
import datetime as dt
import time
import pyprind as bar
from dateutil import parser
from config import cur_lay_api

In [2]:
currencyDate = []
currencyId = []
currencyTitle = []
currencyConv = []
currencyVal = []
cur_con_pfx = "USD"
cur_conv = {"KPW":"North Korean Won",
            "KRW":"South Korean Won",
            "GBP":"British Pound Sterling",
            "EUR":"Euro",
            "HKD":"Hong Kong Dollar"}

#### Manually coded the Currency Names for simplicity
```python
currency_titles = pd.read_html("cur_table.html",encoding="latin-1")[0:]
currencyTitles = currency_titles[0].set_index(["Code", "Name"])
currencyTitles.head()
```

In [3]:
start = dt.date(2018,5,15)
today = dt.date.today()
#end = dt.date(today.year,today.month,today.day-2)
end = dt.date(2018,6,26)
duration = (end - start).days

In [4]:
run_s = time.time()
for cur in cur_conv.keys():
    days = 0
    progress = bar.ProgBar(duration, monitor=True, title="\n\n\n\nRetrieving historical currency data for {0}".format(cur))
    while days <= duration:
        date_it = (start+dt.timedelta(days=(days))).strftime("%Y-%m-%d")
        base_url = "https://apilayer.net/api/historical?access_key={0}&date={1}&source=USD&format=1".format(cur_lay_api,date_it)
        response = requests.get(base_url)
        data = response.json()
        currencyId.append(cur)
        currencyTitle.append(cur_conv[cur])
        currencyConv.append("USD")
        currencyDate.append(parser.parse(date_it))
        currencyVal.append(float(data["quotes"][cur_con_pfx+cur]))
        days+=1
        progress.update()
run_e = time.time()
total_time = run_e - run_s
print("\n\n\nAPI Retrieval Completed in {:0.4f} seconds".format(total_time))





Retrieving historical currency data for KPW
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:14




Retrieving historical currency data for KRW
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:13




Retrieving historical currency data for GBP
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:14




Retrieving historical currency data for EUR
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:14




Retrieving historical currency data for HKD
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:14





API Retrieval Completed in 72.4133 seconds


In [5]:
col_dtypes = {"date":"datetime64[ns]",
              "currency":"object",
              "currencyName":"object",
              "toCurrency":"object",
              "rate":"float64"}

currency_df = pd.DataFrame(np.column_stack([currencyDate, currencyId, currencyTitle, currencyConv, currencyVal]),
                           columns=["date","currency","currencyName","toCurrency","rate"]).astype(col_dtypes)


In [6]:
currency_df.tail()

,date,currency,currencyName,toCurrency,rate
210,2018-06-22,HKD,Hong Kong Dollar,USD,7.845040
211,2018-06-23,HKD,Hong Kong Dollar,USD,7.845040
212,2018-06-24,HKD,Hong Kong Dollar,USD,7.846203
213,2018-06-25,HKD,Hong Kong Dollar,USD,7.846598
214,2018-06-26,HKD,Hong Kong Dollar,USD,7.849004


In [7]:
currency_df["valueUSD"]=1/currency_df["rate"]

In [8]:
currency_df.to_csv("currencyAnalysis.csv")

In [9]:
currencyRatePivot = pd.pivot_table(currency_df, values=["rate"], index=["date"], columns=["currency"])

In [10]:
#currencyRatePivot.drop(index=dt.date(2018,6,27), inplace=True)

In [11]:
currencyUSDPivot = pd.pivot_table(currency_df, values=["valueUSD"], index=["date"], columns=["currency"])

In [12]:
currencyUSDPivot

valueUSD                                        
currency         EUR       GBP       HKD       KPW       KRW
date                                                        
2018-05-15  1.182595  1.349746  0.127392  0.001111  0.000927
2018-05-16  1.181896  1.354701  0.127397  0.001111  0.000929
2018-05-17  1.179522  1.351205  0.127394  0.001111  0.000926
2018-05-18  1.177296  1.346892  0.127392  0.001111  0.000926
2018-05-19  1.177296  1.346892  0.127392  0.001111  0.000926
2018-05-20  1.177026  1.347782  0.127415  0.001111  0.000926
2018-05-21  1.179386  1.343039  0.127426  0.001111  0.000929
2018-05-22  1.179110  1.343976  0.127401  0.001111  0.000931
2018-05-23  1.171232  1.337043  0.127425  0.001111  0.000929
2018-05-24  1.172607  1.338294  0.127427  0.001111  0.000927
2018-05-25  1.166583  1.329858  0.127479  0.001111  0.000929
2018-05-26  1.166583  1.329858  0.127479  0.001111  0.000929
2018-05-27  1.169183  1.330867  0.127477  0.001111  0.000932
2018-05-28  1.162929  1.331274  0.127474  0.001111  0.000932
2018-05-29  1.154199  1.325469  0.127462  0.001111  0.000923
2018-05-30  1.166850  1.328533  0.127440  0.001111  0.000931
2018-05-31  1.169589  1.328974  0.127454  0.001111  0.000927
2018-06-01  1.166583  1.334508  0.127491  0.001111  0.000934
2018-06-02  1.166583  1.334508  0.127491  0.001111  0.000934
2018-06-03  1.167134  1.335399  0.127463  0.001111  0.000935
2018-06-04  1.170417  1.331664  0.127455  0.001111  0.000935
2018-06-05  1.172882  1.340659  0.127432  0.001111  0.000935
2018-06-06  1.178553  1.341976  0.127436  0.001111  0.000939
2018-06-07  1.179658  1.341760  0.127453  0.001111  0.000934
2018-06-08  1.177019  1.340411  0.127479  0.001111  0.000932
2018-06-09  1.177019  1.340411  0.127479  0.001111  0.000932
2018-06-10  1.178412  1.341382  0.127448  0.001111  0.000931
2018-06-11  1.177822  1.337292  0.127452  0.001111  0.000930
2018-06-12  1.175066  1.337417  0.127462  0.001111  0.000927
2018-06-13  1.180780  1.338348  0.127431  0.001111  0.000922
2018-06-14  1.157004  1.325609  0.127402  0.001111  0.000917
2018-06-15  1.160491  1.328586  0.127426  0.001111  0.000908
2018-06-16  1.160491  1.328586  0.127426  0.001111  0.000908
2018-06-17  1.159689  1.327475  0.127405  0.001111  0.000906
2018-06-18  1.163336  1.325856  0.127421  0.001111  0.000906
2018-06-19  1.159418  1.317454  0.127400  0.001111  0.000900
2018-06-20  1.158216  1.317263  0.127455  0.001111  0.000903
2018-06-21  1.160900  1.324837  0.127464  0.001111  0.000902
2018-06-22  1.165447  1.326471  0.127469  0.001111  0.000900
2018-06-23  1.165447  1.326471  0.127469  0.001111  0.000900
2018-06-24  1.166447  1.326330  0.127450  0.001111  0.000901
2018-06-25  1.170408  1.327863  0.127444  0.001111  0.000898
2018-06-26  1.165367  1.322087  0.127405  0.001111  0.000895

In [13]:
curRateDeviation = currencyRatePivot.std(axis=0)
print("Currency Exchange Rate Deviation\n------------------------------------\n"+str(curRateDeviation))

Currency Exchange Rate Deviation
------------------------------------
      currency
rate  EUR          0.005694
      GBP          0.005083
      HKD          0.001875
      KPW          0.000261
      KRW         15.552125
dtype: float64


In [14]:
curRateVariance = currencyRatePivot.var(axis=0)
print("Currency Exchange Rate Variance\n------------------------------------\n"+str(curRateVariance))

Currency Exchange Rate Variance
------------------------------------
      currency
rate  EUR         3.242555e-05
      GBP         2.584172e-05
      HKD         3.513878e-06
      KPW         6.786707e-08
      KRW         2.418686e+02
dtype: float64


In [15]:
curUSDDeviation = currencyUSDPivot.std(axis=0)
print("Currency Rate in USD Deviation\n------------------------------------\n"+str(curUSDDeviation))

Currency Rate in USD Deviation
------------------------------------
          currency
valueUSD  EUR         7.788922e-03
          GBP         9.066859e-03
          HKD         3.044346e-05
          KPW         3.216211e-10
          KRW         1.306693e-05
dtype: float64


In [16]:
curUSDVariance = currencyUSDPivot.var(axis=0)
print("Currency Rate in USD Variance\n------------------------------------\n"+str(curUSDVariance))

Currency Rate in USD Variance
------------------------------------
          currency
valueUSD  EUR         6.066730e-05
          GBP         8.220794e-05
          HKD         9.268044e-10
          KPW         1.034401e-19
          KRW         1.707447e-10
dtype: float64
